# Cross check the constrained attributed warming ensemble with chapter 3

Theme Song: Trail of Fire<br>
Artist: Oceansize<br>
Album: Frames<br>
Released: 2007

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd

from ar6.utils.h5 import *

In [2]:
results = load_dict_from_hdf5('../data_output_large/twolayer_AR6-historical.h5')

In [3]:
results.keys()

dict_keys(['AR6-historical', 'AR6-historical_climuncert', 'remove_aerosol', 'remove_aerosol-cloud_interactions', 'remove_aerosol-cloud_interactions_climuncert', 'remove_aerosol-radiation_interactions', 'remove_aerosol-radiation_interactions_climuncert', 'remove_aerosol_climuncert', 'remove_albedo', 'remove_albedo_climuncert', 'remove_anthro', 'remove_anthro_climuncert', 'remove_bc_on_snow', 'remove_bc_on_snow_climuncert', 'remove_ch4', 'remove_ch4_climuncert', 'remove_co2', 'remove_co2_climuncert', 'remove_contrails', 'remove_contrails_climuncert', 'remove_h2o_stratospheric', 'remove_h2o_stratospheric_climuncert', 'remove_land_use', 'remove_land_use_climuncert', 'remove_n2o', 'remove_n2o_climuncert', 'remove_natural', 'remove_natural_climuncert', 'remove_o3', 'remove_o3_climuncert', 'remove_other_wmghg', 'remove_other_wmghg_climuncert', 'remove_solar', 'remove_solar_climuncert', 'remove_volcanic', 'remove_volcanic_climuncert', 'remove_wmghgs', 'remove_wmghgs_climuncert'])

In [4]:
AR6_forc = {}
forcings = ['co2', 'ch4', 'n2o', 'other_wmghg', 'o3', 'h2o_stratospheric',
            'contrails', 'aerosol-radiation_interactions', 'aerosol-cloud_interactions', 
            'bc_on_snow', 'land_use', 'volcanic', 'solar', 'aerosol', 'anthro', 'natural', 'wmghgs']
for forcing in forcings:
    AR6_forc[forcing] = np.zeros(5)
    AR6_forc[forcing] = np.percentile(        
        (results['AR6-historical_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['AR6-historical_climuncert']['surface_temperature'][100:151].mean(axis=0))-
        (results['remove_%s_climuncert' % forcing]['surface_temperature'][260:270].mean(axis=0) - results['remove_%s_climuncert' % forcing]['surface_temperature'][100:151].mean(axis=0)),
        (5,16,50,84,95)
    )    
# aggregated categories
other_anth_chapter3 = (
    (
        (results['AR6-historical_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['AR6-historical_climuncert']['surface_temperature'][100:151].mean(axis=0))-
        (results['remove_o3_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['remove_o3_climuncert']['surface_temperature'][100:151].mean(axis=0))
    ) +
    (
        (results['AR6-historical_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['AR6-historical_climuncert']['surface_temperature'][100:151].mean(axis=0))-
        (results['remove_h2o_stratospheric_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['remove_h2o_stratospheric_climuncert']['surface_temperature'][100:151].mean(axis=0))
    ) +
    (
        (results['AR6-historical_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['AR6-historical_climuncert']['surface_temperature'][100:151].mean(axis=0))-
        (results['remove_contrails_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['remove_contrails_climuncert']['surface_temperature'][100:151].mean(axis=0))
    ) +
    (
        (results['AR6-historical_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['AR6-historical_climuncert']['surface_temperature'][100:151].mean(axis=0))-
        (results['remove_aerosol_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['remove_aerosol_climuncert']['surface_temperature'][100:151].mean(axis=0))
    ) +
    (
        (results['AR6-historical_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['AR6-historical_climuncert']['surface_temperature'][100:151].mean(axis=0))-
        (results['remove_land_use_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['remove_land_use_climuncert']['surface_temperature'][100:151].mean(axis=0))
    ) +
    (
        (results['AR6-historical_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['AR6-historical_climuncert']['surface_temperature'][100:151].mean(axis=0))-
        (results['remove_bc_on_snow_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['remove_bc_on_snow_climuncert']['surface_temperature'][100:151].mean(axis=0))
    )
)
AR6_forc['aerosols_o3_luc_other_anth'] = np.percentile(other_anth_chapter3, (5,16,50,84,95))

AR6_forc['total'] = np.percentile(results['AR6-historical_climuncert']['surface_temperature'][260:270].mean(axis=0) - results['AR6-historical_climuncert']['surface_temperature'][100:151].mean(axis=0), (5,16,50,84,95))

In [5]:
data_output = []
for species in ['anthro','natural','wmghgs','aerosols_o3_luc_other_anth']:
    data_output.append([
        species,
        AR6_forc[species][0],
        AR6_forc[species][2],
        AR6_forc[species][4]
    ])

In [6]:
df = pd.DataFrame(data_output, columns=['Species', 'pc05', 'pc50', 'pc95'])
df.set_index('Species', inplace=True)
df.to_csv('../data_output/fig3.7_data.csv')
df

,pc05,pc50,pc95
Species,,,
anthro,0.823021,1.066305,1.353390
natural,0.042832,0.073580,0.119196
wmghgs,0.993865,1.341781,1.836139
aerosols_o3_luc_other_anth,-0.628487,-0.269288,-0.026619


In [7]:
data_output = []
for species in ['co2', 'ch4', 'n2o', 'other_wmghg', 'o3', 'h2o_stratospheric',
            'contrails', 'aerosol-radiation_interactions', 'aerosol-cloud_interactions', 
            'bc_on_snow', 'land_use', 'volcanic', 'solar']:
    data_output.append([
        species,
        AR6_forc[species][0],
        AR6_forc[species][2],
        AR6_forc[species][4]
    ])
df = pd.DataFrame(data_output, columns=['Species', 'pc05', 'pc50', 'pc95'])
df.set_index('Species', inplace=True)
df.to_csv('../data_output/figSPM_A_data.csv')
df

,pc05,pc50,pc95
Species,,,
co2,0.610586,0.835915,1.162161
ch4,0.160734,0.231712,0.331250
n2o,0.060613,0.083410,0.114195
other_wmghg,0.132641,0.188547,0.264082
o3,0.093972,0.199228,0.335713
h2o_stratospheric,-0.000642,0.020904,0.046858
contrails,0.007677,0.021815,0.039767
aerosol-radiation_interactions,-0.287221,-0.133788,-0.024158
aerosol-cloud_interactions,-0.675327,-0.335482,-0.105885
